In [82]:
#!/usr/bin/env python3
"""
Module Docstring
"""

__author__ = "Your Name"
__version__ = "0.1.0"
__license__ = "MIT"

import argparse
import os
from logzero import logger
from espn_api.football import League


POSITIONS = ["QB", "WR", "TE", "RB"]

""" Main entry point of the app """
logger.info("hello world from the notebook")
logger.info(args)


# Get environment variables
swid = os.getenv('SWID')
espn_s2 = os.environ.get('ESPN_S2')

if swid == None or espn_s2 == None:
    print("SWID or ESPN_S2 environment variables empty. Exiting...")

league = League(league_id=761056, year=2022, espn_s2=espn_s2,swid=swid)


## Get Box Scores, pass in week 
week = 2
if args.week != None:
    week = args.week


print("Week: ", week)
box_scores = league.box_scores(2)



# for matchup in box_scores:
#     print("")
#     print("Home: ", matchup.home_team, " vs. Away: ", matchup.away_team)
#     print("=========================")
#     print("||  ", matchup.home_score, "   ||   ", matchup.away_score)
#     print("=========================")



# ## Team(J.G.'s  ZZ's ) -> J.G.'s  ZZ's 
# def prettyPrintTeam(s):
#     print(s[s.find('(')+1:])
    
# for team in league.teams:
#     print(prettyPrintTeam(str(team.team_name)))
#     # for player in team.Roster:
#     #     player.Name


[I 220924 16:08:28 3687429929:19] hello world from the notebook
[I 220924 16:08:28 3687429929:20] Namespace(arg='/Users/jacobelias/Library/Jupyter/runtime/kernel-340c986b-9a7c-4f18-bb1b-95c776280545.json', flag=True, ame=None, week=None, verbose=0)


Week:  2


In [80]:
def biggestBlowOut(league, week):
    print("Biggest Blowouts: ")
    outcomes = []
    
    box_scores = league.box_scores(week)
    
    for matchup in box_scores:
        ## For Each Matchip calculate the diff
        diff = 0
        if matchup.home_score >= matchup.away_score:
            diff = matchup.home_score - matchup.away_score
            outcomes.append((matchup.home_team, matchup.away_team, diff))
        else:
            diff = matchup.away_score - matchup.home_score
            outcomes.append((matchup.away_team, matchup.home_team, diff))
            
    sorted_scores = sorted(outcomes, key=lambda tup: tup[2], reverse=True)
    print("Blowout Rankings: ", sorted_scores)
    return sorted_scores


biggestBlowOut(league, week)



Biggest Blowouts: 
Blowout Rankings:  [(Team(El Chapo), Team(Kupp My  Bells Gurley), 42.7), (Team(Grayson BeatYu), Team(Chiraqui Soldiers), 42.099999999999994), (Team(Gisele'd in my  Wentz), Team(Top G), 24.30000000000001), (Team(Begelbaum Donuts), Team(Rosen Sandler), 10.200000000000003), (Team(J.G.'s  ZZ's ), Team(The  Red Army), 6.200000000000003)]


[(Team(El Chapo), Team(Kupp My  Bells Gurley), 42.7),
 (Team(Grayson BeatYu), Team(Chiraqui Soldiers), 42.099999999999994),
 (Team(Gisele'd in my  Wentz), Team(Top G), 24.30000000000001),
 (Team(Begelbaum Donuts), Team(Rosen Sandler), 10.200000000000003),
 (Team(J.G.'s  ZZ's ), Team(The  Red Army), 6.200000000000003)]

In [81]:
def biggestBenchWarmer(league, week, position):
    if position not in POSTIONS:
        return []
    
    
    benchWarmers = []
    box_scores = league.box_scores(week)
    for matchup in box_scores:
        ## For Each Matchip calculate the diff
        for player in matchup.home_lineup:
            if player.position == position and player.slot_position == "BE":
#                 print("player: ", player.name, " position: ", player.position, " ", player.slot_position, " ", matchup.home_team, " ", player.points)
                benchWarmers.append((player.points,player.name,player.position,matchup.home_team))
                
                
        for player in matchup.away_lineup:
            if player.position == position and player.slot_position == "BE":
#                 print("player: ", player.name, " position: ", player.position, " ", player.slot_position, " ", matchup.away_team, " ",player.points )
                benchWarmers.append((player.points,player.name,player.position,matchup.away_team))
    

    return sorted(benchWarmers, key=lambda tup: tup[0], reverse=True)


def prettyPrintBenchWarmers(benchWarmers, position):
    print("=============== ", position, " ===============")
    for index, warmer in enumerate(benchWarmers):
        print(index+1,"", warmer[0], " ", warmer[1], " ", warmer[3])
    print("  ")
        

### QBs
qbWarmers = biggestBenchWarmer(league, week, "QB")
prettyPrintBenchWarmers(qbWarmers[:5], "QB")

### RBs
rbWarmers = biggestBenchWarmer(league, week, "RB")
prettyPrintBenchWarmers(rbWarmers[:5], "RB")

### TEs
teWarmers = biggestBenchWarmer(league, week, "TE")
prettyPrintBenchWarmers(teWarmers[:5], "TE")
        
    

===============  QB  ===============
1  26.7   Carson Wentz   Team(Gisele'd in my  Wentz)
2  14.4   Marcus Mariota   Team(Rosen Sandler)
3  12.6   Mac Jones   Team(The  Red Army)
4  10.3   Mitch Trubisky   Team(J.G.'s  ZZ's )
5  7.6   Jameis Winston   Team(Top G)
  
===============  RB  ===============
1  17.8   Tony Pollard   Team(Top G)
2  15.7   Damien Harris   Team(Chiraqui Soldiers)
3  14.6   David Montgomery   Team(Kupp My  Bells Gurley)
4  12.5   Breece Hall   Team(Top G)
5  10.7   Darrell Henderson Jr.   Team(El Chapo)
  
===============  TE  ===============
1  14.0   Darren Waller   Team(Gisele'd in my  Wentz)
2  10.7   Dallas Goedert   Team(J.G.'s  ZZ's )
3  4.1   T.J. Hockenson   Team(Gisele'd in my  Wentz)
4  2.8   Dalton Schultz   Team(Begelbaum Donuts)
5  0.0   George Kittle   Team(El Chapo)
  


In [83]:
# ### Total Points Played / Optimal Lineup
# lineup_format = {
#     "QB": 2
#     "RB": 2
#     ""
# }
# def excessBenchPoints(lineup, position):
#     for player in lineup:
#         if player.position == position and player.slot_position == "BE":
# #              print("player: ", player.name, " position: ", player.position, " ", player.slot_position, " ", matchup.home_team, " ", player.points)
#         benchWarmers.append((player.points,player.name,player.position,matchup.home_team))
        
    



# def managerEffencieny(league, week):
    
#     for box_scores = league.box_scores(week)
#         for matchup in box_scores:
#             getOptimalLineup(matchup.home_lineup)
    

In [91]:
def worstWin(league, week):
    box_scores = league.box_scores(week)
    lowestWinningScore = 99999999999
    saved = None
    for matchup in box_scores:
        if matchup.home_score >= matchup.away_score:
            lowestWinningScore = min(matchup.home_score, lowestWinningScore)
        else:
            lowestWinningScore = min(matchup.away_score, lowestWinningScore)
            
        
        if lowestWinningScore == matchup.away_score or lowestWinningScore == matchup.home_score:
            saved = matchup
    print(saved.home_team, saved.home_score, " vs ", saved.away_team, saved.away_score)

    
worstWin(league, week)

Team(J.G.'s  ZZ's ) 89.8  vs  Team(The  Red Army) 83.6


In [93]:
def worstLoss(league, week):
    box_scores = league.box_scores(week)
    higestLoss = 0
    saved = None
    for matchup in box_scores:
        if matchup.home_score >= matchup.away_score:
            higestLoss = max(matchup.away_score, higestLoss)
        else:
            higestLoss = max(matchup.home_score, higestLoss)
            
        
        if higestLoss == matchup.away_score or higestLoss == matchup.home_score:
            saved = matchup
    print(saved.home_team, saved.home_score, " vs ", saved.away_team, saved.away_score)

    
worstLoss(league, week)

Team(Kupp My  Bells Gurley) 119.2  vs  Team(El Chapo) 161.9


In [114]:
def startingLineupAverage(lineup):
    starterOutput = [] 
    for player in lineup:
        if player.slot_position != "BE" and player.slot_position != "IR":
            starterOutput.append(player.points)         
    return sum(starterOutput)/len(starterOutput)


def highestScoringStarter(lineup):
    highestScore = 0
    bestPlayer = None
    for player in lineup:
        if player.slot_position != "BE" and player.slot_position != "IR":
            if player.points > highestScore:
                highestScore = player.points
                bestPlayer = player
    return bestPlayer
    
    
## Team with greatest differences between max player and team average
def topHeavyTeams(league, week):
    allTeams = []
    box_scores = league.box_scores(week)
    for matchup in box_scores:
        
        ## HOME TEAM
        avg = startingLineupAverage(matchup.home_lineup)
        maxPlayer = highestScoringStarter(matchup.home_lineup)
        allTeams.append((matchup.home_team, avg, maxPlayer, maxPlayer.points - avg))
        
        ## AWAY TEAM
        avg = startingLineupAverage(matchup.away_lineup)
        maxPlayer = highestScoringStarter(matchup.away_lineup)
        allTeams.append((matchup.away_team, avg, maxPlayer, maxPlayer.points - avg))
        
    allTeamSorted = sorted(allTeams, key=lambda tup: tup[3], reverse=True)
    return allTeamSorted
                                         

topHeavyList = topHeavyTeams(league, week)

print("======= Top Heavy Teams ========")
for index, team in enumerate(topHeavyList[:3]):
    print(index+1, " ", team[0], team[2].name, team[2].points, "{:.2f}".format(team[1]))
        

======= Top Heavy Teams ========
1   Team(Rosen Sandler) Tua Tagovailoa 38.7 12.12
2   Team(Top G) Stefon Diggs 38.8 13.07
3   Team(El Chapo) Lamar Jackson 42.5 20.24


In [117]:
## Team with greatest differences between max player and team average
def highestTeamAverageForStarters(league, week):
    allTeams = []
    box_scores = league.box_scores(week)
    for matchup in box_scores:
        
        ## HOME TEAM
        avg = startingLineupAverage(matchup.home_lineup)
        allTeams.append((matchup.home_team, avg))
        
        ## AWAY TEAM
        avg = startingLineupAverage(matchup.away_lineup)
        maxPlayer = highestScoringStarter(matchup.away_lineup)
        allTeams.append((matchup.away_team, avg))
        
    allTeamSorted = sorted(allTeams, key=lambda tup: tup[1], reverse=True)
    return allTeamSorted


print("======= Highest Average for Starters ========")
for index, team in enumerate(highestTeamAverageForStarters(league,week)[:3]):
    print(index+1, " ", team[0], team[1])
                                   

======= Highest Average for Starters ========
1   Team(El Chapo) 20.2375
2   Team(Gisele'd in my  Wentz) 16.1125
3   Team(Grayson BeatYu) 15.375
